In [3]:
import numpy as np
import scipy.stats as st
import scipy.sparse as spa
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from numpy.linalg import inv

In [7]:
# Discretizing time and space

T = 2
dt = 0.01
t_grid = np.arange(0, T+dt, dt)

prodmax = 6
consmax = 6
h = 0.2
x_grid = np.arange(-prodmax, consmax+h, h)

In [8]:
v_true = np.zeros((len(t_grid), len(x_grid)))

for n, t in enumerate(t_grid):
    v_true[n,:] = t*x_grid**2        
        
# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=v_true.T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [9]:
# Heat equation - Implicit, forwards in time

v_fdm_impl = np.zeros((len(t_grid), len(x_grid)))
v_fdm_impl[:, 0] = v_true[:, 0]
v_fdm_impl[:, -1] = v_true[:, -1]

mu = 1 # diffusion rate

# Create and Invert matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
M_inv = inv(np.eye(len(x_grid[1:-1]))-dt*mu/h**2*A)
#print(np.linalg.cond(M_inv))

for l, t in enumerate(t_grid[:-1]):
    # Compute and update forcing term
    f = np.array([x**2-2*t_grid[l+1]*mu for x in x_grid[1:-1]])       
    f[0] = f[0] + mu/h**2*v_fdm_impl[l+1, 0]
    f[-1] = f[-1] + mu/h**2*v_fdm_impl[l+1, -1]
    
    # Compute next layer
    v_fdm_impl[l+1, 1:-1] = M_inv@(v_fdm_impl[l, 1:-1]+dt*f)

# Create 3D error plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_impl).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [10]:
# Heat equation - Explicit, backwards in time

# starts well then blows up

v_fdm_epx_back = np.zeros((len(t_grid), len(x_grid)))
v_fdm_epx_back[:, 0] = v_true[:, 0]
v_fdm_epx_back[:, -1] = v_true[:, -1]
v_fdm_epx_back[-1, :] = v_true[-1, :] # Terminal condition

mu = -1 # diffusion rate

# Create and Invert matrix
up_diag = np.ones(len(x_grid[2:-1]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))-dt*mu/h**2*A
print(np.linalg.cond(M))

for l, t in enumerate(t_grid[1:]):
    # iteration backwards in time
    l_back = len(t_grid)-1-l
    
    # Compute and update forcing term
    f = np.array([x**2-2*t_grid[l_back]*mu for x in x_grid[1:-1]])
    f[0] = f[0] + mu/h**2*v_fdm_epx_back[l_back, 0]
    f[-1] = f[-1] + mu/h**2*v_fdm_epx_back[l_back, -1]
        
    # Compute next layer
    v_fdm_epx_back[l_back-1, 1:-1] = M@v_fdm_epx_back[l_back, 1:-1]-dt*f

# Create 3D error plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_epx_back).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

1458.358423480119


In [11]:
# Heat equation - Explicit, forwards in time

v_fdm_epx = np.zeros((len(t_grid), len(x_grid)))
v_fdm_epx[:, 0] = v_true[:, 0]
v_fdm_epx[:, -1] = v_true[:, -1]

mu = 1 # diffusion rate

# Create and Invert matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))+dt*mu/h**2*A
#print(np.linalg.cond(M))

for l, t in enumerate(t_grid[:-1]):
    # Compute and update forcing term
    f = np.array([x**2-2*t*mu for x in x_grid[1:-1]])       
    f[0] = f[0] + mu/h**2*v_fdm_epx[l, 0]
    f[-1] = f[-1] + mu/h**2*v_fdm_epx[l, -1]
    
    # Compute next layer
    v_fdm_epx[l+1, 1:-1] = M@v_fdm_epx[l, 1:-1]+dt*f

# Create 3D error plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_epx).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [12]:
# Heat equation - Implicit, backwards in time

# Attention as potential stability issues

v_fdm_back_impl = np.zeros((len(t_grid), len(x_grid)))
v_fdm_back_impl[:, 0] = v_true[:, 0]
v_fdm_back_impl[:, -1] = v_true[:, -1]
v_fdm_back_impl[-1, :] = v_true[-1, :]# Terminal condition

mu = -2 # diffusion rate

# Create and Invert matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
M_inv = inv(np.eye(len(x_grid[1:-1]))+dt*mu/h**2*A)
#print(np.linalg.cond(M_inv))

for l, t in enumerate(t_grid[1:]):
    # iteration backwards in time
    l_back = len(t_grid)-1-l
    
    # Compute and update forcing term
    f = np.array([x**2-2*t_grid[l_back-1]*mu for x in x_grid[1:-1]])
    f[0] = f[0] + mu/h**2*v_fdm_back_impl[l_back-1, 0]
    f[-1] = f[-1] + mu/h**2*v_fdm_back_impl[l_back-1, -1]
    
    # Compute next layer
    v_fdm_back_impl[l_back-1, 1:-1] = M_inv@(v_fdm_back_impl[l_back, 1:-1]-dt*f)

# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_back_impl).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")